In [ ]:
from functools import partial
import pandas as pd
import os
from pathlib import Path

from sepal_ui import sepalwidgets as sw
from sepal_ui import mapping as sm
import ipyvuetify as v

from utils import messages as ms
from utils import parameters as pm
from scripts import vizualisation as viz
from scripts import table as table_run

In [ ]:
#input and output as mutable objects 
class Viz_io:
    def __init__(self):
        #input
        self.check = False
        self.sources = None
        self.bands = None
        self.start_year = pm.min_start_year
        self.end_year = pm.max_end_year
        self.square_size = 2000 #in meters

viz_io = Viz_io()

In [ ]:
#output 
viz_output = sw.Alert().add_msg(ms.VIZ_INTRO)

#btn 
viz_btn = sw.Btn(ms.VIZ_BTN)

#create the inputs 
viz_sources = v.Select(items=pm.sources, label=ms.VIZ_SOURCES, v_model=None, multiple=True)
viz_bands = v.Select(items=[*pm.getAvailableBands()], label=ms.VIZ_BANDS, v_model=None)
viz_min_year = v.Select(class_='mr-5 ml-5', items=[i for i in range(pm.min_start_year, pm.max_end_year+1)], label=ms.START_YEAR, v_model=pm.min_start_year)
viz_max_year = v.Select(class_='ml-5 mr-5', items=[i for i in range(pm.min_start_year, pm.max_end_year+1)], label=ms.END_YEAR, v_model=pm.max_end_year)
viz_years = v.Layout(xs=12, row=True,  children=[viz_min_year, viz_max_year])
viz_square_size = v.Slider(step=500, min=pm.min_square, max=pm.max_square, label=ms.SQUARE_SIZE, v_model=2000, thumb_label='always', class_='mt-5')

#bind the inputs
viz_output.bind(viz_sources, viz_io, 'sources')
viz_output.bind(viz_bands, viz_io, 'bands')
viz_output.bind(viz_min_year, viz_io, 'start_year')
viz_output.bind(viz_max_year, viz_io, 'end_year')
viz_output.bind(viz_square_size, viz_io, 'square_size')

#create a band selector 
id_ = 'viz_widget'
title = 'select your vizualisation Parameters'

viz_input = sw.Tile(
    id_,
    title,
    btn=viz_btn,
    inputs=[viz_sources, viz_bands, viz_years, viz_square_size],
    output=viz_output
)

In [ ]:
#vizualise data 
#def process_start(widget, event, data, output, m, maps):
def process_start(widget, event, data, output):
    
    #toggle the loading button
    widget.toggle_loading()
    
    #load the input 
    file = tb_io.file
    pts = tb_io.pts
    bands = viz_io.bands
    sources = viz_io.sources
    start = viz_io.start_year
    end = viz_io.end_year
    square_size = viz_io.square_size
    
    #check input
    if not output.check_input(file, ms.NO_PTS): return widget.toggle_loading()
    if not output.check_input(bands, ms.NO_BANDS): return widget.toggle_loading()
    if not output.check_input(sources, ms.NO_SOURCES): return widget.toggle_loading()
    if not output.check_input(start, ms.NO_START): return widget.toggle_loading()
    if not output.check_input(end, ms.NO_END): return widget.toggle_loading()
    if not output.check_input(square_size, ms.NO_SQUARE): return widget.toggle_loading()
    if start > end:
        output.add_msg(ms.WRONG_DATES, 'error')
        return widget.toggle_loading()
    
    #security when user remove all satellites (sources = [])
    if not len(sources) > 0: 
        output.add_msg(ms.NO_SOURCES, 'error')
        return widget.toggle_loading()
    
    #generate a sum-up of the inputs
    msg = viz.set_msg(pts, bands, ' & '.join(sources), Path(file).stem, start, end, square_size)
    
    output.add_msg(msg, 'warning')
    
    #change the checked value 
    viz_io.check = True
    
    #toggle the loading button
    widget.toggle_loading()
    
    return 

viz_btn.on_event('click', partial(
    process_start,
    output = viz_output
))   

In [ ]:
viz_input